In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
import keras.models
import numpy as np
import pandas as pd
import string
import nltk
from keras.applications.densenet import layers
from keras.regularizers import l2
from keras_preprocessing.sequence import pad_sequences
from sklearn.naive_bayes import MultinomialNB,CategoricalNB
from keras_preprocessing.text import Tokenizer
from matplotlib import pyplot as plt
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
import re
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
def clean(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    number = re.compile(r"[0-9]")
    speacial_f = re.compile(r"\x89Û_")
    speacial_s = re.compile(r"\x89ÛÒ")
    speacial_t = re.compile(r"\x89ÛÏ")
    first_row = url.sub(r"", text)
    second_row = number.sub(r"",first_row)
    third_row = speacial_f.sub(r"",second_row)
    fourth_row = speacial_s.sub(r"",third_row)
    return speacial_t.sub(r"", fourth_row)

def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

def load_data():
    train_raw_data = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
    test_raw_data = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
    return train_raw_data['text'], train_raw_data['target'], test_raw_data

def URLandpunct(data):
    data = data.map(clean)
    data = data.map(remove_punct)
    return data

def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stopwords.words('english')]
    return " ".join(filtered_words)
if __name__ == '__main__':
    train_raw_X, train_raw_Y, expriment = load_data()
    exprimentX = expriment['text']     
    #eliminating URL and others
    # train
    trainX = URLandpunct(train_raw_X)
    exprimentX = URLandpunct(exprimentX)
    #stopwords
    trainX = trainX.map(remove_stopwords)
    exprimentX = exprimentX.map(remove_stopwords)
    #vectorization
    tfidf = TfidfVectorizer(sublinear_tf=True, max_features=60000, min_df=1, norm='l2',  ngram_range=(1,2))
    features = tfidf.fit_transform(trainX)
    features_test = tfidf.transform(exprimentX)
    #Getting test set
    #split data 
    skf = StratifiedKFold(n_splits=5, random_state=50, shuffle=True)
    accuracy = [] 
    n = 1
    predictions = pd.DataFrame()
    for train_idx, test_idx in skf.split(features,train_raw_Y):
        X_train,X_val=features[train_idx],features[test_idx]
        y_train,y_val=train_raw_Y.iloc[train_idx],train_raw_Y.iloc[test_idx]
        #model= LogisticRegression(max_iter=1000,C=3)
        model=MultinomialNB(alpha=0.5)
        #model=svm.SVC(max_iter=1000)
        model.fit(X_train,y_train)
        s = model.predict(X_val)
        accuracy.append(accuracy_score(y_val, s))
        predictions[str(n)] = model.predict(features_test) 
        n+=1
    print(np.array(accuracy).mean())
    df=predictions[['1','2','3','4','5']].mode(axis=1)
    expriment['target']=df[0]
    print(expriment)
    expriment.to_csv("mysubmission.csv",index=False)     

0.8008668657458179
         id keyword location  \
0         1     NaN      NaN   
1         4     NaN      NaN   
2         5     NaN      NaN   
3         6     NaN      NaN   
4         7     NaN      NaN   
...     ...     ...      ...   
7608  10869     NaN      NaN   
7609  10870     NaN      NaN   
7610  10871     NaN      NaN   
7611  10872     NaN      NaN   
7612  10873     NaN      NaN   

                                                   text  target  
0     Our Deeds are the Reason of this #earthquake M...       1  
1                Forest fire near La Ronge Sask. Canada       1  
2     All residents asked to 'shelter in place' are ...       1  
3     13,000 people receive #wildfires evacuation or...       1  
4     Just got sent this photo from Ruby #Alaska as ...       1  
...                                                 ...     ...  
7608  Two giant cranes holding a bridge collapse int...       1  
7609  @aria_ahrary @TheTawniest The out of control w...       1  
76

In [3]:
import keras.models
import numpy as np
import pandas as pd
import string
import nltk
from keras.applications.densenet import layers
from keras.regularizers import l2
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
from matplotlib import pyplot as plt
from nltk.corpus import stopwords
import re

# total words
from sklearn.model_selection import train_test_split

max_words = 10000
# text lenth
maxlen = 100
#output dimension
embedding_dim = 128

def clean(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    number = re.compile(r"[0-9]")
    speacial_f = re.compile(r"\x89Û_")
    speacial_s = re.compile(r"\x89ÛÒ")
    speacial_t = re.compile(r"\x89ÛÏ")
    first_row = url.sub(r"", text)
    second_row = number.sub(r"",first_row)
    third_row = speacial_f.sub(r"",second_row)
    fourth_row = speacial_s.sub(r"",third_row)
    return speacial_t.sub(r"", fourth_row)

def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

def load_data():
    train_raw_data = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
    test_raw_data = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
    return train_raw_data['text'], train_raw_data['target'], test_raw_data

def URLandpunct(data):
    data = data.map(clean)
    data = data.map(remove_punct)
    return data

def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stopwords.words('english')]
    return " ".join(filtered_words)


def tokenized(dataframe):
    # 1、Processing
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(dataframe)
    vocab = len(tokenizer.word_index)+1
    train_X = tokenizer.texts_to_sequences(dataframe)
    # 2、padding
    train_X = pad_sequences(train_X, maxlen=maxlen)
    return train_X, vocab
if __name__ == '__main__':
    print("Loading")
    trainX, trainY, expriment = load_data()
    exprimentX = expriment['text']
    #eliminating URL and others
    # train
    trainX = URLandpunct(trainX)
    exprimentX = URLandpunct(exprimentX)
    #stopwords
    trainX = trainX.map(remove_stopwords)
    exprimentX = exprimentX.map(remove_stopwords)
    #Tokenizing
    trainX, train_words = tokenized(trainX)
    exprimentX, train_words = tokenized(exprimentX)
    #Getting test set
    trainX, testX, trainY, testY = train_test_split(trainX, trainY, test_size=0.2)
    #Modelling
    model = keras.models.Sequential()
    model.add(layers.Embedding(max_words, embedding_dim, input_length=maxlen))
    model.add(layers.LSTM(32, dropout=0.20))
    model.add(layers.Dense(1, activation="sigmoid", kernel_regularizer=l2(0.2)))
    # model.summary()
    model.compile(optimizer="adam",
                  loss="BinaryCrossentropy",
                  metrics=['accuracy'],
    )
    history = model.fit(
        trainX,
        trainY,
        epochs=20,
        validation_data=(testX, testY)
    )
    score = model.evaluate(testX, testY, verbose=0)
    exprimentY = model.predict(exprimentX)
    predictions = [1 if p > 0.5 else 0 for p in exprimentY]
    print(score[1])
    print("Complete")
    print(exprimentY)
    submission = pd.DataFrame({'id': expriment['id'].values.tolist(), 'target': predictions})
    submission.to_csv('submission.csv', index=False)
    #visulization
    #accuracy
    # acc = history.history['accuracy']
    # val_acc = history.history['val_accuracy']
    # loss = history.history['loss']
    # val_loss = history.history['val_loss']
    # plt.subplot(1, 2, 1)
    # plt.plot(acc, label='Training Accuracy')
    # plt.plot(val_acc, label='Validation Accuracy')
    # plt.title('Training and Validation Accuracy')
    # plt.legend()
    # #loss
    # loss = history.history['loss']
    # val_loss = history.history['val_loss']
    # plt.subplot(1, 2, 2)
    # plt.plot(loss, label='Training Loss')
    # plt.plot(val_loss, label='Validation Loss')
    # plt.title('Training and Validation Loss')
    # plt.legend()
    # plt.show()



Loading



User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_P

Epoch 1/20
191/191 [==============================] - 24s 111ms/step - loss: 0.7494 - accuracy: 0.7049 - val_loss: 0.5511 - val_accuracy: 0.8043
Epoch 2/20
191/191 [==============================] - 21s 108ms/step - loss: 0.4381 - accuracy: 0.8635 - val_loss: 0.5167 - val_accuracy: 0.8050
Epoch 3/20
191/191 [==============================] - 20s 103ms/step - loss: 0.3918 - accuracy: 0.8882 - val_loss: 0.5385 - val_accuracy: 0.7984
Epoch 4/20
191/191 [==============================] - 21s 109ms/step - loss: 0.3242 - accuracy: 0.9251 - val_loss: 0.5504 - val_accuracy: 0.7951
Epoch 5/20
191/191 [==============================] - 20s 103ms/step - loss: 0.2838 - accuracy: 0.9433 - val_loss: 0.5742 - val_accuracy: 0.7873
Epoch 6/20
191/191 [==============================] - 20s 106ms/step - loss: 0.2637 - accuracy: 0.9521 - val_loss: 0.5980 - val_accuracy: 0.7859
Epoch 7/20
191/191 [==============================] - 21s 109ms/step - loss: 0.2469 - accuracy: 0.9583 - val_loss: 0.5925 - val_ac